In [1]:
from joblib import dump, load
import pandas as pd
import numpy as np
from headers import *
from sklearn.metrics import classification_report, make_scorer, confusion_matrix
from IPython.display import Markdown, display
from math import sqrt

Mean return for purchases (1 and 2-year holding period) that beat benchmark

In [2]:
df = pd.read_csv('../data/combined_inner.csv')
for label in yAlpha:
    positiveAlpha = df[df[label] >= 0]
    print(f'{label}: {round(positiveAlpha[label].mean(),3)}, overall mean: {round(df[label].mean(),3)}, median: {round(df[label].median(),3)}')


alpha1Year: 0.255, overall mean: 0.025, median: -0.004
adjustedAlpha1Year: 0.255, overall mean: 0.027, median: -0.002
equalAlpha1Year: 0.246, overall mean: -0.001, median: -0.026
equalAdjustedAlpha1Year: 0.245, overall mean: -0.001, median: -0.026
alpha2Year: 0.456, overall mean: 0.035, median: -0.048
adjustedAlpha2Year: 0.452, overall mean: 0.037, median: -0.045
equalAlpha2Year: 0.46, overall mean: -0.004, median: -0.091
equalAdjustedAlpha2Year: 0.457, overall mean: -0.004, median: -0.09


Note - 2 year returns are 2 year returns, not CAGR over 2 years. Can't be compared 1 to 1

In [3]:

for label in yAlpha:
    model = load('./models2/' + label + '.joblib')
    testedData = pd.read_csv('../data/svm_results/' + label + '.csv')

    X = testedData[ratioKeys + relativeRatioKeys]
    # TODO: adjust/test cut off for another svm batch - DONE in general_learning.ipynb
    y = np.where(testedData[label] <= 0, 0, 1)
    display(Markdown(f'**Results for key: {label}**'))
    print(f'data count: {testedData[label].count()}')
    print(f'data mean: {testedData[label].mean()}')
    print(f'data median: {testedData[label].median()}')

    print('====TEST=====')
    y_pred = model.predict(X)
    xTaken = X[y_pred == 1]
    results = testedData.loc[testedData.index.intersection(xTaken.index)]

    print(f'res count {results[label].count()}')
    print(f'res mean {results[label].mean()}')
    print(f'res median {results[label].median()}')

    print(confusion_matrix(y, y_pred))

    display(Markdown(f'**diff: {results[label].mean() - testedData[label].mean()}**'))
    if '2' in label:
        cagrToBenchmark = sqrt(results[label].mean() - testedData[label].mean() + 1) - 1
        print(f'CAGR: {cagrToBenchmark}')
    print('-------------------------------------------------')

# TODO: iterate over y cutoffs. Train each cutoff with gridCV. Train linearSVC first, and check how faster it is. 
# TODO: maybe use (Linear)SVR - regression, since we have linear regression

**Results for key: alpha1Year**

data count: 6053
data mean: 0.0300172769962029
data median: -0.0031354597826141
====TEST=====
res count 3191
res mean 0.08618092754640873
res median 0.0552135037242193
[[1746 1311]
 [1116 1880]]


**diff: 0.05616365055020583**

-------------------------------------------------


**Results for key: adjustedAlpha1Year**

...

CAGR: 0.1652585306336345
-------------------------------------------------


**Results for key: equalAdjustedAlpha2Year**

data count: 6053
data mean: 0.002947332607081462
data median: -0.0894219371820089
====TEST=====
res count 668
res mean 0.3558874474355833
res median 0.16869667708090824
[[3380  219]
 [2005  449]]


**diff: 0.35294011482850185**

CAGR: 0.1631595397143515
-------------------------------------------------
